In [ ]:
import pandas as pd
import numpy as np 
from sqlalchemy import func, create_engine
#from faker import Faker
import re, json, logging , html, os
import openpyxl
from datetime import date, timedelta
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_parser import issue_parser
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.dbupdate_parser import dbupdate_parser
from lib.script_generator import script_generator 
from lib.db_parser import db_parser   
from bs4 import BeautifulSoup    
#from nltk.corpus import stopwords 
#sw=stopwords.words('english') 
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore') 



# PIPELINE

### Requirements: Jira / PDF / DOC / etc.. 
### SpreadSheet 
### SQL 
### Code / Forms

---

### Jira Parse All Tickets

In [ ]:
sql= """
    SELECT CAST(PK_Question AS NVARCHAR(9)) [{PK_Question}] 
    , CAST(QGROUP AS NVARCHAR(9)) [{FK_QuestionGroup}] 
    ,SortPos [{SortPos}] 
    ,FK_PickListType [{FK_PickListType}] 
    , FK_QuestionType [{FK_QuestionType}] 
    ,IdText [{IdText}] 
    , QText AS [{QText}]
    , help_text FROM vwQuestions WHERE PK_FORM = '2023-A-SAOP'
""" 
#print(sql)
df = sql_todf(sql,  ctx.connstr) 
df

In [ ]:
gen=script_generator(ctx)
code=gen.generate(df, code_template_path=f'{ctx.get_tempalte_dir()}fsma_QuestionsInsert.sql')  
print( code ) 

In [ ]:

ip = issue_parser(ctx)     
df=ip.parse('8838:8843' )  
df 

In [ ]:
ip = issue_parser(ctx)     
df=ip.parse( '9600:9605' )  
df

##### DataFrame to Code

In [ ]:
gen=script_generator(ctx)

In [ ]:
gen=script_generator(ctx)
sql=gen.generate(df, code_template_path=f'{ctx.get_tempalte_dir()}aspx\\ig.sql')
print(sql) 

In [ ]:
form_code=script_generator(ctx).generate(df) 
print(form_code) 

### Forms from Database Records

In [ ]:
parser = db_parser(ctx)
df=parser.parse(question_group='5141')
df

In [ ]:
df.columns

In [ ]:
gen=script_generator(ctx)
sql=gen.generate(df, code_template_path=f'{ctx.get_tempalte_dir()}fsma_QuestionsInsert.sql')
with open(fr"{ctx.get_dest()}\script.sql", 'w', encoding='UTF-8') as fw:  
    fw.write(sql)

In [ ]:
form_code=script_generator(ctx).generate(df) 
print(form_code)
with open(fr"{ctx.get_dest()}\script.aspx", 'w', encoding='UTF-8') as fw: 
    form_code =   form_code.replace('\ufeff', '')
    fw.write(form_code)

In [ ]:
gen=script_generator(ctx)
code=gen.generate(df, code_template_path=f'{ctx.get_tempalte_dir()}aspx\\ig.sql')
print(code)

### NLP

In [ ]:
picklist_string = 'Date Determined / TBD / Continuous ATO'  
pl=picklist_recommender(ctx, use_cache=False).recommend(picklist_string.split(' / '), generate_sql=True, threshhold=(.825, .7))
pl 

##### PARSE DB UPDATE SCRIPT 

In [ ]:
parser = dbupdate_parser(ctx, path=r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\_3_DBUpdate7.66_CIO_Q1_2025.sql') 
df=parser.parse() 
df['FK_QuestionGroup'] = df['section'].apply(lambda s: f"@PK_QGroup+{s}" ) 
s=parser.to_sql() 
with open(fr"{ctx.get_dest()}\script.sql", 'w', encoding='UTF-8') as fw: 
    fw.write(s) 

In [ ]:
df10 = df.loc[ df["FK_QuestionGroup"] == '@PK_QGroup+13']
df10['sortpos']=df10['sortpos'] 
df10 

In [ ]:
print( parser.to_sql()  )

In [ ]:
parser = db_parser(ctx)
df=parser.parse(question_group='4530')
s=df_to_sql(df,ctx)
print(s)

In [ ]:
class aspx_generator():
    def __init__(self, ctx):  
        self.ctx=ctx 
    def generate(self, PK_FORM='2023-A-HVA', classname=None, where=None, commit=False, PageNameFilter='~', dest=None ):
        parser=db_parser(ctx)
        generator=script_generator(ctx)
        sql= f"""
            SELECT DISTINCT PK_FORM, PK_QuestionGroup, SectionNum, GroupName, ASPX
            FROM vwQuestions  WHERE 1=1 AND PK_FORM='{PK_FORM}' 
            --AND {where}
            ORDER BY PK_QuestionGroup ASC
        """
        if classname == None: classname=PK_FORM.replace('-','_')
        if where != None: sql=sql.replace('--AND', 'AND  ')
        #print(sql)
        df=sql_todf(sql, self.ctx.connstr)     
        df['PNAME']=df['ASPX'].apply(lambda s: s.split('/')[len(s.split('/'))-1].replace('.aspx',''))
        pages=df.to_dict(orient='records')    
        src=ctx.get_tempalte_dir()
        out=''
        for p in pages:   
            PageName = re.sub(PageNameFilter,'',f"{classname}_{p['GroupName']}")
            for ext in ['.aspx','.aspx.designer.vb','.aspx.vb']:
                with open(f'{src}{classname}{ext}', 'r', encoding='UTF-8', errors='ignore') as fr: 
                    txt = fr.read().encode(  'utf-8', errors='ignore' ).decode('utf-8')
                    txt=txt.replace('{classname}',PageName) 
                    txt=txt.replace('{PK_key}',f"{p['PK_QuestionGroup']}")
                    txt=txt.replace('{GroupName}',f"{p['GroupName']}")  
                    if ext=='.aspx': 
                        parsed=parser.parse(question_group=p['PK_QuestionGroup'])
                        code = generator.generate(parsed)
                        txt=txt.replace(f'<!--controls-->',code) 
                    out=out+txt
                path=f"{dest}{p['PNAME']}{ext}"
                print(path)
                if commit:  
                    with open(path, 'w', encoding='UTF-8') as fw:
                        fw.write(txt)
        with open(fr"{ctx.get_dest()}\script.aspx", 'w', encoding='UTF-8') as fw: 
            fw.write(out)

dest=rf"C:\\temp\\"  
gen = aspx_generator(ctx)
gen.generate(PK_FORM='2025-A-HVA', dest=dest)

 